In [25]:
import math
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

# Preprocessing Documents

## Reading Documents

In [26]:
def ReadingFromFile():
    FilesvsLines={}
    NumOfFile=int(input("Enter Number Of Files You Will Enter : "))
    for x in range(0,NumOfFile):
        NameOfFile=input("Enter Name of file : ")
        File=open(NameOfFile+".txt",'r')
        Lines = File.read().splitlines()
        FilesvsLines[NameOfFile]=Lines
        File.close()
    return FilesvsLines,NumOfFile

## Splitting Words

In [27]:
def SplitWords(Words):
    TWords=[]
    for Word in Words:
        TWords.extend(word_tokenize(Word))
    return TWords

## Remove Stop Words By NLTK

In [28]:
def RemoveStopWords(words):
    filtered_sentence = []
    for word in words:
        if word not in stop_words:
            filtered_sentence.append(word.capitalize())
    return filtered_sentence

In [29]:
def PreprocessingDocuments():
    FilesvsLines,NumOfDocuments=ReadingFromFile()
    for key ,value in FilesvsLines.items():
        FilesvsLines[key]=RemoveStopWords(SplitWords(value))
    return FilesvsLines,NumOfDocuments

In [30]:
FilesvsLines,NumOfDocuments=PreprocessingDocuments()
print(FilesvsLines)

Enter Number Of Files You Will Enter : 4
Enter Name of file : DOC1
Enter Name of file : DOC2
Enter Name of file : DOC3
Enter Name of file : DOC4
{'DOC1': ['English', 'Tutorial'], 'DOC2': ['Advance', 'English'], 'DOC3': ['English', 'Structure'], 'DOC4': ['Tutorial']}


In [31]:
def create2DArray(FilesvsLines):
    array=[]
    WordsVSTF={}
    for key in FilesvsLines:
        for value in FilesvsLines[key]:
            array.append(value)
    vac=sorted(list(set(array)))
    for  x in vac:
        for value in FilesvsLines.values():
            if(x in value):
                if (x in WordsVSTF.keys()):
                    WordsVSTF[x].append(1)
                else:
                    WordsVSTF[x]=[1]
            else:
                if (x in WordsVSTF.keys()):
                    WordsVSTF[x].append(0)
                else:
                    WordsVSTF[x]=[0]
    return WordsVSTF

In [32]:
WordsVSTF=create2DArray(FilesvsLines)
print(WordsVSTF)

{'Advance': [0, 1, 0, 0], 'English': [1, 1, 1, 0], 'Structure': [0, 0, 1, 0], 'Tutorial': [1, 0, 0, 1]}


In [33]:
def CalculateIDF(WordsVSTFIDF,NumOfDocuments):
    DICTOfIDF={}
    for key,value in WordsVSTFIDF.items():
        sumo=0
        for x in value:
            sumo=sumo+x
        nsumo=NumOfDocuments/sumo
        idf=math.log10(nsumo)
        DICTOfIDF[key]=idf
        value=list(map(lambda x: x*idf,value))
        WordsVSTFIDF[key]=value
    return WordsVSTFIDF,DICTOfIDF

In [34]:
WordsVSTFIDF,DICTOfIDF=CalculateIDF(WordsVSTF,NumOfDocuments)
print(DICTOfIDF)
print(WordsVSTFIDF)

{'Advance': 0.6020599913279624, 'English': 0.12493873660829993, 'Structure': 0.6020599913279624, 'Tutorial': 0.3010299956639812}
{'Advance': [0.0, 0.6020599913279624, 0.0, 0.0], 'English': [0.12493873660829993, 0.12493873660829993, 0.12493873660829993, 0.0], 'Structure': [0.0, 0.0, 0.6020599913279624, 0.0], 'Tutorial': [0.3010299956639812, 0.0, 0.0, 0.3010299956639812]}


In [35]:
def CalculateLengthOfDocuments(WordsVSTFIDF,NumOfDocuments):
    LengthOfDocuments=[]
    for i in range(0,NumOfDocuments):
        sumo=0
        for values in WordsVSTFIDF.values():
            sumo+=math.pow(values[i],2)
        LengthOfDocuments.append(math.sqrt(sumo))
    return LengthOfDocuments

In [36]:
LengthOfDocuments=CalculateLengthOfDocuments(WordsVSTFIDF,NumOfDocuments)
print(LengthOfDocuments)

[0.32592751678054843, 0.6148869172970786, 0.6148869172970786, 0.3010299956639812]


# Query Preprocessing

In [37]:
def QueryPreprocessing(DICTOfIDF):
    Query=[]
    QueryVsTFIDF={}
    query=input("Enter Your Query : ")
    queryterm=RemoveStopWords(query.split(" "))
    listOfIDF=list(DICTOfIDF.keys())
    for x in queryterm:
        if(x in listOfIDF):
            Query.append(x)
    print(Query)        
    UniqueQuery=sorted(list(set(Query)))
    print(UniqueQuery)
    for x in UniqueQuery:
        idf=DICTOfIDF.get(x)
        QueryVsTFIDF[x]=Query.count(x)*idf
    sumo=0    
    for x in QueryVsTFIDF.values():
        sumo+=math.pow(x,2)
    LengthOfQuery=math.sqrt(sumo) 
    return QueryVsTFIDF,LengthOfQuery

In [39]:
QueryVsTFIDF,LengthOfQuery=QueryPreprocessing(DICTOfIDF)
print(QueryVsTFIDF)
print(LengthOfQuery)

Enter Your Query : advance Tutorial
['Advance', 'Tutorial']
['Advance', 'Tutorial']
{'Advance': 0.6020599913279624, 'Tutorial': 0.3010299956639812}
0.673123533571129


In [40]:
def CalculateCosineSimilarity(WordsVSTFIDF,LengthOfDocuments,QueryVsTFIDF,LengthOfQuery,NumOfDocuments):
    CosineSimilarity={}
    for i in range(0,NumOfDocuments):
        sumo=0
        for key ,value in QueryVsTFIDF.items():
            sumo+=WordsVSTFIDF[key][i]*QueryVsTFIDF[key]
            
        cos=sumo/(LengthOfDocuments[i]*LengthOfQuery)
        CosineSimilarity["DOC"+str(i+1)]=cos
    sorted_by_value=sorted(CosineSimilarity.items(), key=lambda CosineSimilarity: CosineSimilarity[1],reverse=True)
    print(sorted_by_value)
    return CosineSimilarity    

In [41]:
CalculateCosineSimilarity(WordsVSTFIDF,LengthOfDocuments,QueryVsTFIDF,LengthOfQuery,NumOfDocuments)

[('DOC2', 0.8757688799495646), ('DOC4', 0.4472135954999579), ('DOC1', 0.4130510613035182), ('DOC3', 0.0)]


{'DOC1': 0.4130510613035182,
 'DOC2': 0.8757688799495646,
 'DOC3': 0.0,
 'DOC4': 0.4472135954999579}